## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,73.36,95,65,7.00,broken clouds
1,1,Norman Wells,CA,65.2820,-126.8329,53.64,58,20,10.36,few clouds
2,2,Yulara,AU,-25.2406,130.9889,62.08,38,86,5.84,overcast clouds
3,3,Khani,GE,41.9563,42.9566,65.77,88,96,0.98,overcast clouds
4,4,Barrow,US,71.2906,-156.7887,35.62,87,0,10.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,57,0,14.97,clear sky
7,7,Malanje,AO,-9.5402,16.3410,79.83,24,82,2.44,broken clouds
8,8,Georgetown,MY,5.4112,100.3354,84.13,86,40,4.61,scattered clouds
12,12,Faanui,PF,-16.4833,-151.7500,80.42,69,8,17.00,clear sky
15,15,Micheweni,TZ,-4.9667,39.8333,77.59,80,79,8.84,broken clouds
23,23,Limbang,MY,4.7500,115.0000,78.91,97,100,2.33,overcast clouds
24,24,Atuona,PF,-9.8000,-139.0333,78.40,64,87,20.09,overcast clouds
32,32,Trinidad,BO,-14.8333,-64.9000,82.38,65,0,6.91,clear sky
35,35,Trenggalek,ID,-8.0500,111.7167,75.70,93,94,1.86,overcast clouds
40,40,Cabedelo,BR,-6.9811,-34.8339,84.87,63,30,9.24,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                253
City                   253
Country                252
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Arraial Do Cabo,BR,75.16,clear sky,-22.9661,-42.0278,
7,Malanje,AO,79.83,broken clouds,-9.5402,16.3410,
8,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,
12,Faanui,PF,80.42,clear sky,-16.4833,-151.7500,
15,Micheweni,TZ,77.59,broken clouds,-4.9667,39.8333,
23,Limbang,MY,78.91,overcast clouds,4.7500,115.0000,
24,Atuona,PF,78.40,overcast clouds,-9.8000,-139.0333,
32,Trinidad,BO,82.38,clear sky,-14.8333,-64.9000,
35,Trenggalek,ID,75.70,overcast clouds,-8.0500,111.7167,
40,Cabedelo,BR,84.87,scattered clouds,-6.9811,-34.8339,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Arraial Do Cabo,BR,75.16,clear sky,-22.9661,-42.0278,Pousada Porto Praia
7,Malanje,AO,79.83,broken clouds,-9.5402,16.3410,Hotel Ginga
8,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,Cititel Penang
12,Faanui,PF,80.42,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
15,Micheweni,TZ,77.59,broken clouds,-4.9667,39.8333,
...,...,...,...,...,...,...,...
661,Altamira,BR,87.69,clear sky,-3.2033,-52.2064,Requinte Hotel
667,Buchanan,LR,78.26,moderate rain,5.8808,-10.0467,Kamaneahn Hotel Inc
668,Mabaruma,GY,86.00,broken clouds,8.2000,-59.7833,Monty Brooms Guest House
669,Brasileia,BR,84.65,scattered clouds,-11.0161,-68.7481,Pousada Tobias


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))